In [70]:
import rasterio
import rasterio.features
import rasterio.warp
import os
import glob
from geojson import Point, Feature, FeatureCollection, dump
import geopandas as gpd
import pandas as pd
import shapely.wkt

In [71]:
path = "/home/simon/CDE_UBS/thesis/data_collection/sen2/merged_reprojected"
footprints_path = "/home/simon/CDE_UBS/thesis/data_collection/footprints/sen2_footprints/"
# define search criteria
search_criteria = "*.tif"
search_term = os.path.join(path, search_criteria)
# perform search
files = glob.glob(search_term)
spot6_footprints_path = ""

In [73]:
dates = []
tiles = []
for i in files:
    dates.append(i[78:86])
    tiles.append(i[102:108])

geoms = []
file_names = []
# Extract Footprints
counter = 0
for i in files:
    # def counter cond
    counter = counter+1
    if counter%1==0:
        perc = 100 * float(counter)/float(len(files))
        print(str(int(perc))+"%","       ",end="\r")
    
    
    file_name = i[i.rfind("/")+1:]
    file_names.append(file_name)
    with rasterio.open(i) as dataset:

        # Read the dataset's valid data mask as a ndarray.
        mask = dataset.dataset_mask()

        # Extract feature shapes and values from the array.
        for geom, val in rasterio.features.shapes(
                mask, transform=dataset.transform):

            # Transform shapes from the dataset's own coordinate
            # reference system to CRS84 (EPSG:4326).
            #geom = rasterio.warp.transform_geom(
            #    dataset.crs, 'EPSG:4326', geom, precision=6)

            # Print GeoJSON shapes to stdout.
            geoms.append(geom)

In [81]:
# Modify WKT to be able to pass into GDF
wkt_geoms = []
for v,i in enumerate(geoms):
    coordinates = i["coordinates"]
    wkt_str = "POLYGON (("
    for coor in coordinates:
        for x in coor:
            wkt_str=wkt_str+""
            x_1 = str(x).replace(",","")
            x_1 = x_1.replace("(","")
            x_1 = x_1.replace(")","")
            #x_1 = x_1.replace("","")
            wkt_str = wkt_str+x_1+","
        wkt_str = wkt_str[:wkt_str.rfind(",")] + wkt_str[wkt_str.rfind(",")+1:]
            
    wkt_str = wkt_str.replace("[","")
    wkt_str = wkt_str.replace("]","")
    wkt_str = wkt_str+"))"
    #print(wkt_str)
    wkt_geoms.append(shapely.wkt.loads(wkt_str))

In [82]:
df = pd.DataFrame()
df["file"] = file_names
df["date"] = dates
df["tile"] = tiles
df["geom"] = wkt_geoms
gdf = gpd.GeoDataFrame(df, geometry=df.geom)
gdf = gdf[["file","date","tile","geometry"]]
gdf.set_crs(epsg=2154)
gdf.to_file(footprints_path+"footprints_sen2_2154.shp")